# Setting up a test subgraph and TRAPI endpoint for KG-Bioportal

## Prepare subgraph for ontologies relevant to clinical data

Load the set of ontologies to work with. This is defined in `clinical_ontologies.yaml`.

In [ ]:
import yaml

In [ ]:
test_ontologies = []
with open ('clinical_ontologies.yaml', 'r') as infile:
    ontologies_dict = yaml.safe_load(infile)

for ontology in ontologies_dict['ontologies']:
    try:
        if ontology['test_set']:
            test_ontologies.append(ontology['name'])
    except KeyError:
        continue

test_ontologies

Now build a graph of these alone. This assumes that the transformed Bioportal graphs are in `../transformed/ontologies/` by default, 

In [ ]:
test_ontologies_str = ",".join(test_ontologies)

In [ ]:
%cd ../
!python run.py catmerge --include_only {test_ontologies_str}

See how the result looks.

In [ ]:
!tar -xvzf data/merged/merged-kg.tar.gz

In [ ]:
!head merged-kg_edges.tsv

In [ ]:
!head merged-kg_nodes.tsv

These didn't really get CURIE-d properly (an issue with the transform not using the expected prefix set) but let's go ahead anyway.

## Prepare as Neo4j

Start a local Neo4j instance first.

Install may look like the directions here:
https://neo4j.com/docs/operations-manual/current/installation/linux/debian/#debian-installation

On my system, I had to enable systemctl first:
https://askubuntu.com/questions/1379425/system-has-not-been-booted-with-systemd-as-init-system-pid-1-cant-operate

On the first run, there may be a message like this:
```bash
$ systemctl start neo4j.service
Failed to start neo4j.service: Interactive authentication required.
See system logs and 'systemctl status neo4j.service' for details.
```

Just run:
```bash
$ sudo systemctl enable neo4j.service
$ sudo systemctl start neo4j.service
$ sudo systemctl status neo4j.service
```

and it should be `active (running)`.

In [ ]:
from kgx.transformer import Transformer

input_args = {
    'filename': ["../merged-kg_edges.tsv",
                 "../merged-kg_nodes.tsv"],
    'format': 'tsv'
}
output_args = {
    'uri': 'neo4j://localhost:7474',
    'username': 'neo4j',
    'password': 'demo',
    'format': 'neo4j'
}

t = Transformer()
t.transform(input_args, output_args)


## Set up and run Plater on its own

See https://github.com/TranslatorSRI/Plater

In [ ]:
!git clone https://github.com/TranslatorSRI/Plater

In [ ]:
!python -m pip install -r Plater/PLATER/requirements.txt

Need a config file available as env variables.
Should look something like this:
```bash
 WEB_HOST=0.0.0.0
 WEB_PORT=8080
 NEO4J_HOST=neo4j
 NEO4J_USERNAME=neo4j
 NEO4J_PASSWORD=demo
 NEO4J_HTTP_PORT=7474
 PLATER_TITLE='Plater'
 PLATER_VERSION='1.2.0-7'
 BL_VERSION='1.6.1'
```

In [ ]:
!source Plater/.env

Running the FastAPI app can be erratic in a notebook, but run this:
```bash
$ uvicorn PLATER.services.server:APP --host 0.0.0.0 --port 8080 --reload
```

## Set up and run Automat